### **Problem 01:**

In [33]:
import pandas as pd
import numpy as np 
import csv
import re



In [34]:
# Hàm fix lỗi dư hoặc thiếu dấu (" ")
def fix_csv_quotes(file_path, output_path):
    with open(file_path, 'r', encoding='utf-8') as infile, open(output_path, 'w', newline='', encoding='utf-8') as outfile:
        reader = infile.readlines()
        writer = csv.writer(outfile)

        for line in reader:
            # Check if the line has mismatched quotes and fix it
            num_quotes = line.count('"')
            if num_quotes % 2 != 0:
                # If odd number of quotes, add a quote at the end
                line = line.strip() + '"\n'
            writer.writerow(next(csv.reader([line])))

In [35]:
# Define the input and output file paths
input_file = '../Data/temp_data.csv'
output_file = '../Data/clean_data.csv'

# Call the function to fix the CSV quotes
fix_csv_quotes(input_file, output_file)


### **Problem 02:**

In [36]:
def clean_string(string):

    """
    Hàm này sẽ làm sạch chuỗi bằng cách loại bỏ các biểu tượng và ký tự không cần thiết.
    (nhưng giữ lại các kí tự chữ (hoa và thường), số, dấu chấm, dấu phẩy, dấu | )
    
    string: Chuỗi cần làm sạch.
    cleanString: Chuỗi đã được làm sạch.

    """

    # Hàm isinstance check xem data input của mình có phải là string không 
    if isinstance(string, str):
        cleanString = re.sub(r'[^\w\s.,|]', '', string) # Xoá tất cả các kí tự ngoại trừ chữ (thường, hoa), số, dấu chấm, dấu phẩy, dấu |
        return cleanString
    else:
        return string

In [37]:
def clean_file_csv(input_csv, output_csv, column_index):

    """
    Hàm này sẽ làm sạch một cột cụ thể trong file CSV, do có những cột khác (VD cột Postdate có dấu : ), 
    Vì vậy nếu apply full file csv nó sẽ fix hết cả những cột không cần thiết -> chỉ áp dụng chính xác cột cần xử lí thôi.

    input_csv (str): Đường dẫn tới file CSV đầu vào (temp_data).
    output_csv (str): Đường dẫn tới file CSV đầu ra (clean_data).
    column_index (int): Chỉ số của cột cần làm sạch.
    """

    # Đọc file CSV vào DataFrame
    df = pd.read_csv(input_csv, on_bad_lines = 'skip')

    # Kiểm tra xem chỉ số cột có hợp lệ không
    if column_index < len(df.columns):
        # Áp dụng hàm làm sạch cho cột cụ thể
        col = df.columns[column_index]
        df[col] = df[col].apply(clean_string)
    else:
        print(f"Invalid column index !")

    # Lưu lại DataFrame đã được làm sạch thành file CSV mới
    df.to_csv(output_csv, index = False)



In [38]:
input_csv = '../Data/temp_data.csv'
output_csv = '../Data/clean_data.csv'
column_index = 8  # Chỉ số của cột cần làm sạch (bắt đầu từ 0), cột 8 là cột Description
clean_file_csv(input_csv, output_csv, column_index)

### **Problem 03:**

In [39]:
# Đọc lại file csv, xoá các dòng bị lỗi 
# (mình nhận thấy số lượng dòng bị lỗi không nhiều ~ 10 dòng nên mình sẽ skip luôn vì tụi mình có nhiều data mà =))))
df = pd.read_csv('../Data/clean_data.csv', on_bad_lines = 'skip')

# Loại bỏ cột No và cột Title không cần thiết 
df = df.drop(columns = df.columns[[0, 2]])

# Thay thế "Thoả thuận" bằng NaN cho dễ xử lí về sau (tạm thời)
df['Price'] = df['Price'].replace('TT', np.nan)

# Thiết lập ngưỡng cho số lượng giá trị NaN tối đa cho phép trong một hàng
max_nan = 3  

# Xóa các hàng có số lượng giá trị NaN lớn hơn ngưỡng
df = df.dropna(thresh = len(df.columns) - max_nan)  # thresh: chỉ định số lượng giá trị NOT NULL cần thiết để giữ lại hàng

# Xử lí cột Price 
# df = df[df['Price'] <= 1000]

# Xử lí cột Postdate (chỉ giữ lại Ngày xoá Giờ)
# Chuyển đổi cột 'Postdate' sang kiểu datetime
df['Postdate'] = pd.to_datetime(df['Postdate'])

# Tạo một cột mới chỉ chứa ngày
df['Date'] = df['Postdate'].dt.date

# Tìm vị trí (index) của cột 'Postdate'
col_index = df.columns.get_loc('Postdate')

# Xóa cột 'Postdate' cũ
df = df.drop(columns=['Postdate'])

# Chèn cột 'Date' mới vào đúng vị trí cũ của cột 'Postdate'
df.insert(col_index, 'Postdate', df.pop('Date'))

# Xoá các hàng bị duplicate
df = df.drop_duplicates()

# Tạo lại cột No
df.insert(0, 'No', range(1, len(df) + 1))

# Load lại những thay đổi vào file clean_data.csv
df.to_csv('../Data/clean_data.csv', index=False)

# Sau khi thực hiện clean bằng tất cả các bước trên thì từ 2000 rows data còn 1843 rows.

In [40]:
import pandas as pd

# Đọc file CSV vào một DataFrame
df = pd.read_csv('input.csv')

# Xóa các hàng mà cả hai cột 'Price' và 'Address' đều có giá trị NaN
df_cleaned = df.dropna(subset=['Price', 'Address'], how='all')

# Lưu lại DataFrame đã xử lý vào file CSV mới
df_cleaned.to_csv('output.csv', index=False)

FileNotFoundError: [Errno 2] No such file or directory: 'input.csv'

In [ ]:
import pandas as pd

# Đọc file CSV vào một DataFrame
df = pd.read_csv('input.csv')

# Giả sử cột chứa giá trị thời gian có tên là 'Timestamp'
# Chuyển đổi cột 'Timestamp' sang kiểu datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Tạo một cột mới chỉ chứa ngày
df['Date'] = df['Timestamp'].dt.date

# Xoá cột 'Timestamp' cũ
df = df.drop(columns=['Timestamp'])

# Lưu lại DataFrame đã xử lý vào file CSV mới
df.to_csv('output.csv', index=False)